In [1]:
!pip install gensim

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 24.0 MB 62.4 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 26.6 MB/s eta 0:00:01


In [2]:
import moxing as mox
mox.file.copy_parallel("obs://lrn-source/QA/data/questions","./data/questions")
mox.file.copy_parallel("obs://lrn-source/QA/data/annotations","./data/annotations")


INFO:root:Using MoXing-v2.0.0.rc2.4b57a67b-4b57a67b
INFO:root:Using OBS-Python-SDK-3.20.9.1


In [3]:
# 
import json
import matplotlib
import numpy as np
import re
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim import corpora
from collections import defaultdict

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
config = {
    # base info about dataset
    'train_num':44375,
    'test_num':21435,
    'test_num':21435,
    'clean_path':'./clean_data/',
    'origin_path':'./data/',
    'clear':False,
    # paradim setting
    'sentence_len':8,
    'use_stopwords':False,

}
stoplist = set('a the this that these those of on in to for'.split())
path = config['clear'] ? config['clean_path'] : config['origin_path']
# load all Q and A
j_eval = open(path+'val.json')
j_train = open(path+'train.json')
j_test = open(path+'test.json')

Q_train = json.load(j_train)['questions']
Q_eval = json.load(j_eval)['questions']
Q_test = json.load(j_test)['questions']

j_eval = open(path+'val.json')
j_train = open(path+'train.json')
j_test = open(path+'test.json')

# not checked ,uncertain about how to get the raw data
A_train = json.load(j_train)['annotations']
A_eval = json.load(j_eval)['annotations']
A_test = json.load(j_test)['annotations']

Q_set = Q_train+Q_eval+Q_test

A_set = A_train+A_eval+A_test
# print(len(Q_set))
# print(len(A_set))
Vocab = []
for i in range(len(Q_set)):
    Q = Q_set[i]['question']
    A = A_set[i]['multiple_choice_answer']
    Q = Q.lower()
    Q = re.sub(r"[^a-zA-Z]+", r" ",Q)
    Q = Q.strip()
    A = A.lower()
    A = re.sub(r"[^a-zA-Z]+", r" ",A)
    A = A.strip()

    Q_vocab = Q.split(' ')
    if(config['use_stopwords']):
        Vocab.append([word for word in Q_vocab if word not in stoplist])
    else:
        Vocab.append(Q_vocab)

    A_vocab = A.split(' ')
    if(config['use_stopwords']):
        Vocab.append([word for word in A_vocab if word not in stoplist])
    else:
        Vocab.append(A_vocab)
        

print(len(Vocab))

174490


In [8]:
# 调用gensim生成词向量
dictionary = corpora.Dictionary(Vocab)   # 生成词典

# 将文档存入字典，字典有很多功能，比如
# diction.token2id 存放的是单词-id key-value对
# diction.dfs 存放的是单词的出现频率
dictionary.save('./pre/vocab.dict')  # store the dictionary, for future reference
corpus = [dictionary.doc2bow(sentence) for sentence in Vocab]
corpora.MmCorpus.serialize('./pre/vocab.mm', corpus)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<2874 unique tokens: ['brushing', 'is', 'she', 'daytime', 'man']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<4033 unique tokens: ['brushing', 'is', 'she', 'daytime', 'man']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<4852 unique tokens: ['brushing', 'is', 'she', 'daytime', 'man']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<5564 unique tokens: ['brushing', 'is', 'she', 'daytime', 'man']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<6134 unique tokens: ['brushing', 'is', 'she', 'daytime', 'man']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<6634 unique tokens: ['brushing', 'is', 'she', 'daytime', 'man']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Dictionary<7078 unique tokens: ['brushing'

In [20]:
# 从磁盘中读取向量化结果？
# corpus = corpora.MmCorpus('/tmp/deerwester.mm')
# corpus

# 尝试用词向量那一章的办法处理词向量
from multiprocessing import cpu_count
model = Word2Vec(Vocab,vector_size=100,window = 3,min_count = 1,workers = cpu_count(),sg = 1)
model.wv.save_word2vec_format('./pre/embedding.txt',binary=False)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 36559 words, keeping 2874 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 73204 words, keeping 4033 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 110073 words, keeping 4852 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 146421 words, keeping 5564 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 182900 words, keeping 6134 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 219494 words, keeping 6634 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 256296 words, keeping 7078 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 293117 words, keeping 7501 word 

In [23]:
word2vec_model = KeyedVectors.load_word2vec_format("./pre/embedding.txt")
word2vec_model['this']
# len(word2vec_model)

INFO:gensim.models.keyedvectors:loading projection weights from ./pre/embedding.txt
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (10408, 100) matrix of type float32 from ./pre/embedding.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-06-23T20:26:08.205713', 'gensim': '4.2.0', 'python': '3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:15:32) \n[GCC 7.5.0]', 'platform': 'Linux-4.19.36-vhulk1907.1.0.h619.eulerosv2r8.aarch64-aarch64-with-centos-2.0-SP8', 'event': 'load_word2vec_format'}


array([-0.43695927,  0.02246003, -0.07530419,  0.13626575, -0.08835112,
       -0.2867401 ,  0.5803464 ,  0.5448465 , -0.6631262 , -0.6201518 ,
       -0.1046695 , -0.7451572 ,  0.03050958,  0.35961044,  0.0805601 ,
       -0.03590161, -0.00731205,  0.24237946,  0.224675  , -0.78133595,
        0.2770713 , -0.11566632,  0.49113053, -0.16143158,  0.2930674 ,
       -0.39127913, -0.32296893,  0.585891  , -0.45273766, -0.42255235,
       -0.23288438, -0.2072499 ,  0.27956203, -0.8619466 , -0.06067633,
       -0.25105014,  0.33284077, -0.13124242,  0.39626262,  0.07168514,
       -0.16977108,  0.03811906, -0.7035129 ,  0.33312336, -0.1725771 ,
       -0.707339  , -0.00395804,  0.31287694,  0.35647744, -0.26511833,
        0.01563093,  0.3163982 ,  0.48337865, -0.17918037,  0.06929579,
       -0.54578614, -0.4041756 , -0.4866031 , -0.0626772 ,  0.19045337,
        0.3096853 , -0.14717278, -0.12583974,  0.06920189, -0.50478804,
        0.37050587,  0.41515544, -0.0781661 , -0.24529983,  0.19

In [22]:
model.wv['many']

array([-0.38685104,  0.7942848 , -0.40107405, -0.39577168,  0.0808575 ,
       -0.6366186 , -0.09438095, -0.2709141 , -0.05271592, -0.568316  ,
        0.27638334, -0.15826419, -0.10832916, -0.55156875,  0.8924032 ,
        0.08380511,  1.2759517 , -0.21557905, -0.48230177, -0.8034091 ,
        0.6037414 ,  0.4913805 ,  0.391892  , -0.00178589,  0.0989664 ,
        0.85308605, -0.8634709 , -0.07041136,  0.06774336,  0.6210676 ,
        0.37937164,  0.24620673,  0.41684696, -0.55675733,  0.39251766,
       -0.5234859 , -0.16858786,  0.74370044,  0.1117577 , -0.7084758 ,
       -0.15516785, -0.23557016, -0.16437747, -0.10147497, -0.7530076 ,
       -0.60166615, -0.46578893, -0.21451098,  0.95491385,  0.724726  ,
        0.10225928, -0.47126597, -0.44990054, -0.47867182,  0.10097928,
       -0.11808329,  0.40420142, -0.11659685, -0.21419919,  1.3122077 ,
       -0.28542975,  0.25742012,  0.69313467, -0.07157668,  0.40163374,
        0.1555124 ,  0.6968507 ,  0.6330399 , -0.4277124 , -0.54